In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [15]:
# <------------------------ Section 1 --------------------------------->
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Run inference on an image
results = model('10619969_366278796862949_668151654247090443_o.jpg')  # results list

# desired_class ids -> this is a list of class ids that you want to consider (the id should be same as it is given in coco)
desired_class_ids=[0,2,3]
desired_class_names=dict(({0,"alpha"},{2,"gama"},{3,"delta"})) # class labels corresponding to desired_class_ids

# for checking the class_id corresponding to class_labels (for coco dataset) you can uncomment the below code
# print(results[0].names)

# <--------------------------- Section 2 ------------------------------------->
# Store desired results only
for r in results:
    # indexes -> list of indexes of only that classes which you want consider
    indexes = [index for index, item in enumerate(r.boxes.cls) if item in desired_class_ids]
    # now we are storing results for classes that you want to consider
    class_ids=r.boxes.cls[indexes] # stores the class_detected (includes only the classes that you want to consider)
    class_confs=r.boxes.conf[indexes]
    boxes_xyxy=r.boxes.xyxy[indexes] # stores the bounding boxes in xyxy format
    boxes_xyxyn=r.boxes.xyxyn[indexes] # stores the bounding boxes in xyxy (normalized) format
    boxes_xywh=r.boxes.xywh[indexes] # stores the bounding boxes in xywh format



image 1/1 /content/10619969_366278796862949_668151654247090443_o.jpg: 640x480 6 persons, 1 skateboard, 11.0ms
Speed: 3.0ms preprocess, 11.0ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 480)


In [26]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!zip -r /content/gpt.zip /content/results/
from google.colab import files
files.download("/content/gpt.zip")


  adding: content/results/ (stored 0%)
  adding: content/results/checkpoint-36985/ (stored 0%)
  adding: content/results/checkpoint-36985/config.json (deflated 51%)
  adding: content/results/checkpoint-36985/training_args.bin (deflated 52%)
  adding: content/results/checkpoint-36985/model.safetensors (deflated 7%)
  adding: content/results/checkpoint-36985/generation_config.json (deflated 24%)
  adding: content/results/checkpoint-36985/optimizer.pt (deflated 8%)
  adding: content/results/checkpoint-36985/scheduler.pt (deflated 56%)
  adding: content/results/checkpoint-36985/trainer_state.json (deflated 80%)
  adding: content/results/checkpoint-36985/rng_state.pth (deflated 25%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:


# <----------------------------- Section 3 ---------------------------------->
# code for visualizing the results
import cv2
import os
from ultralytics.utils.plotting import Annotator,colors
def visualize_detections(img_src, output_dir="runs/", boxes_xyxy=None, class_ids=None, class_confs=None):
    """
    This function visualizes the detections by plotting bounding boxes on the image.

    Args:
    img_src: The input image path.
    output_dir (str, optional): The directory where the output will be saved. Defaults to "runs/".
    boxes_xyxy (list): The list of bounding boxes in the format (x1, y1, x2, y2).
    class_ids (list): The list of class IDs for each detected object.
    class_confs (list): The list of confidence scores for each detected object.

    Returns:
    numpy array: The image with plotted bounding boxes.
    """
    img=cv2.imread(img_src)
    # Create a copy of the image
    imgcpy = img.copy()

    # Initialize the annotator
    annotator = Annotator(imgcpy)

    # Loop over each detected object
    for i in range(len(class_ids)):
        # Add a bounding box and label to the image
        annotator.box_label(box=boxes_xyxy[i], label=(desired_class_names[class_ids[i].item()]+" "+ str(round(class_confs[0].item(), 2))), color=colors(class_ids[i], True))

    # Check if output directory exists
    if not os.path.exists(output_dir):
        # If not, create it
        os.makedirs(output_dir)

    # Save the image
    cv2.imwrite(os.path.join(output_dir, 'predicted_img.jpg'), annotator.result())
    # Return the annotated image
    return annotator.result()

# run the above function to visualize the predited boxes
visualize_detections("10619969_366278796862949_668151654247090443_o.jpg",boxes_xyxy=boxes_xyxy,class_ids=class_ids,class_confs=class_confs)

# now your predicted image is saved at location runs/predicted_img.jpg

array([[[234, 229, 228],
        [233, 228, 227],
        [232, 228, 227],
        ...,
        [237, 237, 237],
        [237, 237, 237],
        [237, 237, 237]],

       [[231, 226, 225],
        [231, 226, 225],
        [229, 225, 224],
        ...,
        [237, 237, 237],
        [237, 237, 237],
        [237, 237, 237]],

       [[227, 223, 222],
        [227, 223, 222],
        [226, 222, 221],
        ...,
        [237, 237, 237],
        [237, 237, 237],
        [237, 237, 237]],

       ...,

       [[255,  42,   4],
        [255,  42,   4],
        [255,  42,   4],
        ...,
        [156, 159, 150],
        [156, 159, 150],
        [156, 159, 150]],

       [[255,  42,   4],
        [255,  42,   4],
        [255,  42,   4],
        ...,
        [156, 159, 150],
        [156, 159, 150],
        [156, 159, 150]],

       [[255,  42,   4],
        [255,  42,   4],
        [255,  42,   4],
        ...,
        [155, 158, 149],
        [155, 158, 149],
        [155, 158, 149]]

In [4]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="coco.yaml", epochs=5, imgsz=640)

Ultralytics YOLOv8.2.72 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line

train: Scanning /content/datasets/coco/labels/train2017.cache... 117266 images, 1021 backgrounds, 0 corrupt: 100%|██████████| 118287/118287 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/datasets/coco/labels/val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train2
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       5000      36335      0.613      0.472      0.506      0.358

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5      3.38G      1.101      1.327      1.163        170        640: 100%|██████████| 7393/7393 [33:57<00:00,  3.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 157/157 [00:26<00:00,  6.02it/s]


                   all       5000      36335      0.619      0.468      0.509      0.361

5 epochs completed in 2.938 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.72 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 157/157 [00:44<00:00,  3.56it/s]


                   all       5000      36335      0.621      0.468      0.509      0.361
                person       2693      10777       0.77      0.656      0.738      0.508
               bicycle        149        314      0.632      0.379      0.439      0.252
                   car        535       1918      0.654      0.513       0.56       0.36
            motorcycle        159        367      0.684      0.559      0.645      0.404
              airplane         97        143       0.76      0.755      0.819      0.641
                   bus        189        283      0.737      0.653      0.735      0.616
                 train        157        190      0.808      0.774      0.834       0.64
                 truck        250        414      0.556      0.377      0.427      0.293
                  boat        121        424      0.551      0.337      0.381       0.21
         traffic light        191        634      0.636      0.334      0.394      0.202
          fire hydran

In [32]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.utils.data import Dataset
import json


tokenizer = GPT2Tokenizer.from_pretrained('gpt2',
                                          bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>',
                                          pad_token='<|pad|>')

In [33]:
print("The max model length is {} for this model".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))



The max model length is 1024 for this model
The beginning of sequence token <|startoftext|> token has the id 50257
The end of sequence token <|endoftext|> has the id 50256
The padding token <|pad|> has the id 50258


In [3]:
import json
import random
# Load your full dataset
with open('custom_dataset.json', 'r') as file:
    dataset = json.load(file)

# Randomly sample a subset of the dataset
subset_size = int(len(dataset) * 0.02)  # Adjust the fraction as needed
subset = random.sample(dataset, subset_size)

# Save the subset
with open('subset_dataset.json', 'w') as file:
    json.dump(subset, file, indent=4)





In [46]:
# Load the dataset
with open('subset_dataset.json', 'r') as f:
    train_data = json.load(f)

with open('custom_eval_dataset.json', 'r') as f:
    val_data = json.load(f)

# Function to format the objects and coordinates
def format_objects(objects):
    formatted_objects = []
    for obj in objects:
        label = obj['label']
        coordinates = ', '.join(map(str, obj['coordinates']))
        formatted_objects.append(f"{label} with coordinates [{coordinates}]")
    return 'The objects detected in the image are: ' + ', '.join(formatted_objects) + '. Describe the scene.'


In [47]:
# Prepare the training data
train_texts = [format_objects(item['objects']) for item in train_data]
train_descriptions = [item['description'] for item in train_data]

# Prepare the validation data
val_texts = [format_objects(item['objects']) for item in val_data]
val_descriptions = [item['description'] for item in val_data]


In [59]:
max_length = 512
#  Tokenize inputs and labels
train_inputs = tokenizer(train_texts, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')
train_labels = tokenizer(train_descriptions, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')

# Prepare TensorDataset and DataLoader
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels['input_ids'])
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)


val_inputs = tokenizer(val_texts, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')
val_labels = tokenizer(val_descriptions, return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')

val_dataset = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_labels['input_ids'])
val_dataloader = DataLoader(val_dataset, batch_size=4)

# Create DataLoaders


In [63]:
from torch.utils.data import DataLoader, TensorDataset
# Create DataLoaders
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_inputs['labels'])
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

val_dataset = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_inputs['labels'])
val_dataloader = DataLoader(val_dataset, batch_size=4)

KeyError: 'labels'

In [51]:
import numpy as np
import random

from transformers import GPT2Tokenizer,GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments
# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))
# Tell pytorch to run this model on the GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# This step is optional but will enable reproducible runs.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [52]:


# We wil create a few variables to define the training parameters of the model
# epochs are the training rounds
# the warmup steps are steps at the start of training that are ignored
# every x steps we will sample the model to test the output

epochs = 10
warmup_steps = 1e2
sample_every = 100



In [53]:


from transformers import AdamW
# AdamW is a class from the huggingface library, it is the optimizer we will be using, and
# we will only be instantiating it with the default parameters.

optimizer = AdamW(model.parameters(),
                  lr = 5e-4,
                  eps = 1e-8
                )



In [54]:
from transformers import get_linear_schedule_with_warmup

"""
Total training steps is the number of data points, times the number of epochs.
Essentially, epochs are training cycles, how many times each point will be seen by the model.
"""

total_steps = len(train_dataloader) * epochs

"""
We can set a variable learning rate which will help scan larger areas of the
problem space at higher LR earlier, then fine tune to find the exact model minima
at lower LR later in training.
"""
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

In [62]:
import random
import time
import datetime
import torch

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

# Set total training start time
total_t0 = time.time()

# Initialize the list to track training statistics
training_stats = []

# Move model to the device
model = model.to(device)

for epoch_i in range(epochs):
    print(f'Beginning epoch {epoch_i + 1} of {epochs}')
    
    # Set epoch start time
    t0 = time.time()

    # Reset the total loss for this epoch
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_masks = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Debugging: Print shapes of input and labels to ensure they match
        print(f"Step {step}: Input IDs shape: {b_input_ids.shape}, Labels shape: {b_labels.shape}")

        # Clear previously calculated gradients
        model.zero_grad()

        # Perform a forward pass
        outputs = model(input_ids=b_input_ids, attention_mask=b_masks, labels=b_labels)
        loss = outputs.loss

        # Accumulate the training loss
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Print sample output every 100 batches
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'Batch {step} of {len(train_dataloader)}. Loss: {batch_loss}. Time: {elapsed}')

            model.eval()
            sample_outputs = model.generate(
                bos_token_id=random.randint(1, tokenizer.vocab_size),
                do_sample=True,
                top_k=50,
                max_length=200,
                top_p=0.95,
                num_return_sequences=1
            )
            for i, sample_output in enumerate(sample_outputs):
                print(f'Example output: {tokenizer.decode(sample_output, skip_special_tokens=True)}')

            model.train()

        # Perform a backward pass to calculate the gradients
        loss.backward()

        # Update parameters and learning rate
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all batches
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took
    training_time = format_time(time.time() - t0)
    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')

    # ==========================
    #       Validation
    # ==========================

    # Set the model to evaluation mode
    model.eval()

    total_eval_loss = 0

    # Evaluate data for one epoch
    for batch in val_dataloader:
        b_input_ids = batch[0].to(device)
        b_masks = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(input_ids=b_input_ids, attention_mask=b_masks, labels=b_labels)
            loss = outputs.loss

        # Accumulate the validation loss
        batch_loss = loss.item()
        total_eval_loss += batch_loss

    # Calculate the average loss over all validation batches
    avg_val_loss = total_eval_loss / len(val_dataloader)

    # Measure how long the validation took
    validation_time = format_time(time.time() - t0)
    print(f'Validation Loss: {avg_val_loss}. Validation Time: {validation_time}')

    # Record all statistics from this epoch
    training_stats.append({
        'epoch': epoch_i + 1,
        'Training Loss': avg_train_loss,
        'Validation Loss': avg_val_loss,
        'Training Time': training_time,
        'Validation Time': validation_time
    })

print(f'Total training took {format_time(time.time() - total_t0)}')

# Save the trained model
model.save_pretrained('fine_tuned_gpt2')
tokenizer.save_pretrained('fine_tuned_gpt2')

# Save training statistics
import pandas as pd
df_stats = pd.DataFrame(training_stats)
df_stats.to_csv('training_stats.csv', index=False)


Beginning epoch 1 of 10
Step 0: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 1: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 2: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 3: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 4: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 5: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 6: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 7: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 8: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 9: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 10: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size([4, 512])
Step 11: Input IDs shape: torch.Size([4, 512]), Labels shape: torch.Size(

RuntimeError: The size of tensor a (2048) must match the size of tensor b (4) at non-singleton dimension 0

In [32]:
def generate_description(objects):
    input_text = format_objects(objects)
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    
    with torch.no_grad():
        output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)
    
    description = tokenizer.decode(output[0], skip_special_tokens=True)
    return description

# Example usage
new_objects = [
    {
        "label": "person",
        "coordinates": [227.74, 187.5, 77.16, 171.12]
    },
    {
        "label": "kite",
        "coordinates": [399.06, 127.59, 58.23, 32.3]
    }
]

print(generate_description(new_objects))


Generate a description of a scene with: wine glass at coordinates [139.67, 279.84, 77.97, 91.24], wine glass at coordinates [83.87, 336.24, 123.04, 87.76], cake at coordinates [285.53, 208.66, 354.47, 210.09], dining table at coordinates [0.0, 392.83, 252.2, 25.33]. more " it  also which would far or most than rather approach man purchasing and ref single ( one trans tried the first second class macro mis, other type thinking to person very was third early de whole character young is


In [65]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load the fine-tuned model
model_path = 'fine_tuned_gpt2'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def format_objects(objects):
    formatted_objects = []
    for obj in objects:
        label = obj['label']
        coordinates = ', '.join(map(str, obj['coordinates']))
        formatted_objects.append(f"{label} with coordinates [{coordinates}]")
    return 'The objects detected in the image are: ' + ', '.join(formatted_objects) + '. Describe the scene.'

def generate_description(model, tokenizer, objects):
    input_text = format_objects(objects)
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    
    with torch.no_grad():
        output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)
    
    description = tokenizer.decode(output[0], skip_special_tokens=True)
    return description

# Example usage
new_objects = [
    {"label": "person", "coordinates": [227.74, 187.5, 77.16, 171.12]},
    {"label": "kite", "coordinates": [399.06, 127.59, 58.23, 32.3]}
]

description = generate_description(model, tokenizer, new_objects)
print(description)


/Users/Apple/anaconda3/envs/DeepLearing/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/Apple/anaconda3/envs/DeepLearing/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:433: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


TypeError: sequence item 54: expected str instance, NoneType found

In [6]:
import torch
print(torch.cuda.get_device_properties(0).total_memory)

42481811456
